In [1]:
# import libraries 
import os 
import numpy as np 
import pandas as pd 
import cv2
import gc
from tqdm import tqdm 
from glob import glob

# Step 1 and 2 
1. collect all data 
2. labelling 

In [2]:
dirs = os.listdir('data')


print(dirs)

['Mask', 'Mask_Chin', 'Mask_Mouth_Chin', 'Mask_Nose_Mouth']


In [3]:
# folder = 'Mask'

images_path = []
labels = []
for folder in dirs:
    path = glob('./data/{}/*.jpg'.format(folder))
    label =['{}'.format(folder)]*len(path)
    #append
    images_path.extend(path)
    labels.extend(label)

# print(path)

In [4]:
# print(label)

# Step 3 
Face Detection 

In [6]:
img_path = images_path[1]
img = cv2.imread(img_path)

In [7]:
# cv2.imshow('original',img)
# cv2.waitKey()
# cv2.destroyAllWindows()

In [8]:
print(img)

[[[  1   3   3]
  [  0   2   2]
  [  0   2   2]
  ...
  [  0   3   8]
  [  0   3   8]
  [  0   3   8]]

 [[  1   3   3]
  [  0   2   2]
  [  0   2   2]
  ...
  [  0   3   8]
  [  0   3   8]
  [  0   3   8]]

 [[  1   3   3]
  [  0   2   2]
  [  0   2   2]
  ...
  [  0   3   8]
  [  0   3   8]
  [  0   3   8]]

 ...

 [[  6  26 111]
  [  6  26 111]
  [  6  26 111]
  ...
  [ 83 130 211]
  [ 83 130 211]
  [ 80 127 208]]

 [[  6  26 111]
  [  6  26 111]
  [  6  26 111]
  ...
  [ 81 128 209]
  [ 82 129 210]
  [ 80 127 208]]

 [[  6  26 111]
  [  6  26 111]
  [  6  26 111]
  ...
  [ 80 127 208]
  [ 81 128 209]
  [ 80 127 208]]]


In [9]:
cv2.imshow('yes',img)
cv2.waitKey()
cv2.destroyAllWindows()

In [10]:
# face detection 
face_detection_model = cv2.dnn.readNetFromCaffe('./models/deploy.prototxt.txt',
                                                './models/res10_300x300_ssd_iter_14000_fp16.caffemodel')

In [11]:
def face_detection_dnn(img):

    #blob from image (rgb mean subtraction image )
    image = img.copy()
    h,w = image.shape[:2]

    blob = cv2.dnn.blobFromImage(image,1,(300,300),(104,117,123),swapRB=True)

    #get the detections 
    face_detection_model.setInput(blob)
    detections = face_detection_model.forward()
    # print(detections)

    for i in range(0,detections.shape[2]):
        confidence = detections[0,0,i,2] #confidence score
    #     print(confidence)
        if confidence > 0.5:
            box = detections[0,0,i,3:7]*np.array([w,h,w,h])
            box = box.astype(int)
            print(box)
            pt1 = (box[0],box[1])
            pt2 = (box[2],box[3])
    #         cv2.rectangle(image,pt1,pt2,(0,255,0),2)
            roi = image[box[1]:box[3],box[0]:box[2]]
        
            return roi
    
    return None
        
        

        


In [12]:
img_roi = face_detection_dnn(img)


[216 142 802 801]


In [13]:
cv2.imshow('roi',img_roi)
cv2.imshow('original',img)
cv2.waitKey(0)
cv2.destroyAllWindows

<function destroyAllWindows>

# Step 5 
Blob from image 

In [14]:
def datapreprocess(img):

    # blob from image(rgb mean subtraction image)
    face = face_detection_dnn(img)
    
    if face is not None:

        #computing blob from image
        blob = cv2.dnn.blobFromImage(face,1,(100,100),(104,117,123),swapRB=True)

        blob_squeeze = np.squeeze(blob).T  # making it from 4 dimension to 3
        blob_rotate = cv2.rotate(blob_squeeze,cv2.ROTATE_90_CLOCKWISE)
        blob_flip = cv2.flip(blob_rotate,1)
        #remove negative values and normalize 
        img_norm = np.maximum(blob_flip,0)/blob_flip.max()
        
        return img_norm
    else:
        return None



   


# apply data preprocessing on all the images

In [15]:
len(images_path)

10000

In [16]:
data_img = []
label_img = []
i=0

for path,label in tqdm(zip(images_path,labels),desc='preprocessing'):
    img = cv2.imread(path)
    process_img = datapreprocess(img)
    if process_img is not None:
        data_img.append(process_img)
        label_img.append(label)
        
    i+=1
    if i%100 == 0:
        gc.collect()

preprocessing: 3it [00:00, 22.12it/s]

[110 139 757 825]
[216 142 802 801]
[203 168 771 860]
[205 132 836 830]
[273 189 805 825]


preprocessing: 9it [00:00, 24.90it/s]

[252 182 758 859]
[314 157 930 849]
[126 175 788 804]
[184 117 785 818]
[ 51 127 771 875]
[168 125 807 798]


preprocessing: 15it [00:00, 25.34it/s]

...

[293 233 850 952]
[251 195 771 898]
[216 218 759 938]
[ 91  83 754 921]
[269 177 829 909]


preprocessing: 9995it [07:25, 21.45it/s]

[233 196 822 945]
[260 192 873 930]
[163 190 756 913]
[210 239 768 832]
[ 267  103 1008  887]


preprocessing: 10000it [07:25, 22.43it/s]

[302 201 853 931]
[274 102 983 877]
[233 204 775 948]


In [19]:
X = np.array(data_img)
y = np.array(label_img)

In [20]:
X.shape , y.shape

((9959, 100, 100, 3), (9959,))

In [21]:
np.savez('./data/data_preprocess.npz',X,y)